#Further data processing and feature development

In [1]:
import numpy as np
import pandas as pd
%pylab inline
import seaborn as sns
from pandasql import sqldf
import matplotlib.pyplot as plt
from ggplot import *
import qgrid
qgrid.nbinstall()
pysqldf = lambda q: sqldf(q, globals())

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Read in training dataset
training_data = pd.read_csv('train.csv', sep = ',')

#fix column names to make life easier...
training_data.columns = ['id', 'survived', 'class', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked']

#add column to indicate it is the training set
training_data['type'] = 'train'

In [3]:
#Read in training dataset
test_data = pd.read_csv('test.csv', sep = ',')

#fix column names to make life easier...
test_data.columns = ['id','class', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked']

#add column to indicate it is the test set
test_data['type'] = 'test'

###Combine the two datasets to create a master version

In [4]:
dataset = training_data.append(test_data)

In [5]:
dataset.shape

(1309, 13)

###Splitting Name to get Surname and Title

Parse surname from name

In [6]:
def get_surname(row):
    return row['name'].split(',')[0].strip()
dataset['surname'] = dataset.apply(get_surname, axis = 1)

Parse title from name

In [7]:
def get_title(row):
    return row['name'].split(',')[1].rsplit('.')[0].strip()
dataset['title'] = dataset.apply(get_title, axis = 1)

In [8]:
def fix_titles(row):
    if row['title'] in ['Dr', 'Col', 'Major', 'Capt', 'Don', 'Jonkheer', 'Sir']:
        return 'Mr'
    elif row['title'] in ['Mlle', 'Ms', 'Dona', 'Lady', 'Mme', 'the Countess']:
        return 'Mrs'
    else:
        return row['title']

dataset['title'] = dataset.apply(fix_titles, axis = 1)

In [9]:
query = 'select title, count(1) as ct from dataset group by title order by ct desc'
pysqldf(query)

,title,ct
0,Mr,775
1,Miss,260
2,Mrs,205
3,Master,61
4,Rev,8


Fix titles:

In [10]:
query = 'select title, count(1) as ct from dataset group by title order by ct desc'
pysqldf(query)

,title,ct
0,Mr,775
1,Miss,260
2,Mrs,205
3,Master,61
4,Rev,8


Improved age imputation strategy:

In [11]:
ages = pd.DataFrame(dataset.groupby(['class', 'title']).median()['age']).reset_index()
print 'Median ages of passengers by class and title:'
ages

Median ages of passengers by class and title:


,class,title,age
0,1,Master,6.0
1,1,Miss,30.0
2,1,Mr,42.0
3,1,Mrs,45.0
4,2,Master,2.0
5,2,Miss,20.0
6,2,Mr,30.0
7,2,Mrs,30.0
8,2,Rev,41.5
9,3,Master,6.0


In [12]:
impute_ages_query = '''
SELECT
    (CASE WHEN t.age > 0 THEN t.age ELSE a.age END) AS age,
    t.cabin,
    t.class,
    t.embarked,
    t.fare,
    t.id,
    t.name,
    t.parch,
    t.sex,
    t.sibsp,
    t.survived,
    t.ticket,
    t.type,
    t.surname,
    t.title
FROM dataset t
LEFT JOIN ages a on a.class = t.class and a.title = t.title
'''
dataset = pysqldf(impute_ages_query)
dataset.head(n = 10)

,age,cabin,class,embarked,fare,id,name,parch,sex,sibsp,survived,ticket,type,surname,title
0,22,None,3,S,7.2500,1,"Braund, Mr. Owen Harris",0,male,1,0,A/5 21171,train,Braund,Mr
1,38,C85,1,C,71.2833,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,female,1,1,PC 17599,train,Cumings,Mrs
2,26,None,3,S,7.9250,3,"Heikkinen, Miss. Laina",0,female,0,1,STON/O2. 3101282,train,Heikkinen,Miss
3,35,C123,1,S,53.1000,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,female,1,1,113803,train,Futrelle,Mrs
4,35,None,3,S,8.0500,5,"Allen, Mr. William Henry",0,male,0,0,373450,train,Allen,Mr
5,26,None,3,Q,8.4583,6,"Moran, Mr. James",0,male,0,0,330877,train,Moran,Mr
6,54,E46,1,S,51.8625,7,"McCarthy, Mr. Timothy J",0,male,0,0,17463,train,McCarthy,Mr
7,2,None,3,S,21.0750,8,"Palsson, Master. Gosta Leonard",1,male,3,0,349909,train,Palsson,Master
8,27,None,3,S,11.1333,9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,female,0,1,347742,train,Johnson,Mrs
9,14,None,2,C,30.0708,10,"Nasser, Mrs. Nicholas (Adele Achem)",0,female,1,1,237736,train,Nasser,Mrs


**Repeat Processing from Notebook 1 to add dummy variables for sex, embarked and cabin.**

In [13]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['embarked'], prefix = 'embarked_at')], axis = 1)
dataset = pd.concat([dataset, pd.get_dummies(dataset['sex'])], axis = 1)
dataset = pd.concat([dataset, pd.get_dummies(dataset['class'], prefix = 'class')], axis = 1)


def cabin_status(row):
    if row['cabin'] == None:
        return 0
    else:
        return 1
    
dataset['in_cabin'] = dataset.apply(cabin_status, axis = 1)   

**Calculate family size from parch and sibsp and create 'family_id' from surname**

In [14]:
dataset['family_size'] = dataset.apply(lambda row: row['parch'] + row['sibsp'] + 1, axis=1)

In [15]:
dataset['family_id'] = dataset.apply(lambda row: row['surname'] + '-' + str(row['family_size']), axis = 1)

**Quick check to see if ticket details can add info to family_id - i.e. people travelling together that don't have the same surname. Looks like this isn't going to be useful**

In [16]:
query = 'select ticket, family_id,  count(1) as ct from dataset group by ticket, family_id order by ct desc'
qgrid.show_grid(pysqldf(query), remote_js=True)

In [17]:
query = 'select family_id from dataset group by family_id'
x = pysqldf(query)
x = x['family_id'].reset_index()
x.columns = ['family_id_int', 'family_id']


In [18]:
query = 'select d.*, x.family_id_int from dataset d left join x on x.family_id = d.family_id'
dataset = pysqldf(query)

**Add dummy variable for title**

In [19]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['title'])], axis = 1)

**Deal with single missing fare value and write Outputs to CSV**

In [29]:
dataset['fare'] = dataset['fare'].fillna(value = 0)
dataset['farebin'] = dataset.apply(lambda row: floor(row['fare'] / 5), axis = 1)

In [30]:
dataset.to_csv('proc_data.csv', sep = ',', index = False)

**More Data Checking........**

In [21]:
b = dataset[['type','age', 'class', 'fare', 'family_size', 'embarked_at_Q', 'embarked_at_S', 'female', 'in_cabin', 'Master', 'Miss', 'Mrs', 'Rev']]
qgrid.show_grid(b, remote_js=True)

In [28]:
dataset.count()

age              1309
cabin             295
class            1309
embarked         1307
fare             1309
id               1309
name             1309
parch            1309
sex              1309
sibsp            1309
survived          891
ticket           1309
type             1309
surname          1309
title            1309
embarked_at_C    1309
embarked_at_Q    1309
embarked_at_S    1309
female           1309
male             1309
class_1.0        1309
class_2.0        1309
class_3.0        1309
in_cabin         1309
family_size      1309
family_id        1309
family_id_int    1309
Master           1309
Miss             1309
Mr               1309
Mrs              1309
Rev              1309
farebin          1309
dtype: int64